In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0`
import $ivy.`sh.almond::almond-spark:0.10.1`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = SparkSession
      .builder()
      .config("spark.sql.shuffle.partitions", 4)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/05/04 21:45:05 INFO SparkContext: Running Spark version 3.2.0
22/05/04 21:45:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/04 21:45:05 INFO ResourceUtils: ==============================================================
22/05/04 21:45:05 INFO ResourceUtils: No custom resources configured for spark.driver.
22/05/04 21:45:05 INFO ResourceUtils: ==============================================================
22/05/04 21:45:05 INFO SparkContext: Submitted application: 03502f9d-23c5-4eca-9d60-88dfd7818c97
22/05/04 21:45:05 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/05/0

import $ivy.$                                  

import $ivy.$                               


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@695b7668
import spark.implicits._



In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [3]:
val races = spark.read.parquet("../../../data/parquetnopart/races.parquet")
    .select("raceId", "year")

races: DataFrame = [raceId: string, year: string]

In [4]:
val driverInfo = spark.read.parquet("../../../data/parquetnopart/drivers.parquet")
    .select(col("driverId"), 
        concat(col("forename"), lit(" "), col("surname")).alias("fullName"))

driverInfo: DataFrame = [driverId: string, fullName: string]

In [5]:
val driverSeasonWindow = Window.partitionBy("driverId", "year")

driverSeasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@8a59592

In [6]:
val driverWindow = Window.partitionBy("driverId")

driverWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1fbcf78e

In [7]:
val seasonWindow = Window.partitionBy("year")

seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@656cb1a

A calcular:
 - Porcentaje de salidas en la primera fila (primero o segundo)
 - Número de salidas en primera fila consecutivas
 - Porcentaje de victorias
 - Veces en el top 2 del campeonato mundial relativo al número de campeonatos en los que compitió
 - Porcentaje de vueltas lideradas
 - Posición media en parrilla (eliminar outliers)
 - Porcentaje de temporadas en las que ha hecho pole
 - Porcentaje de temporadas en las que ganó

In [8]:
val lastRace = Window.partitionBy("year")

val lastRaces = spark.read.parquet("../../../data/parquetnopart/races.parquet")
    .withColumn("round", col("round").cast(IntegerType))
    .withColumn("max", max(col("round")).over(lastRace))
    .where(col("round") === col("max"))
    .select("raceId", "year")

lastRace: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@592bef8f
lastRaces: DataFrame = [raceId: string, year: string]

In [9]:
val driverConstSeasonMap = spark.read.parquet("../../../data/parquetnopart/drivers_constr_season.parquet")

driverConstSeasonMap: DataFrame = [year: string, driverId: string ... 1 more field]

In [10]:
val teammateWindow = Window.partitionBy("year", "constructorId")

teammateWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3cb270dd

In [11]:
val driverDomination = spark.read.parquet("../../../data/parquetnopart/driver_standings.parquet")
    .join(lastRaces, Seq("raceId"), "right")
    .join(driverConstSeasonMap, Seq("driverId", "year"), "left")
    .withColumn("teamPointsPerc", col("points") / sum(col("points")).over(teammateWindow))
    .withColumn("bestOfTeam", max(col("teamPointsPerc")).over(teammateWindow))
    .withColumn("dominatedTeammate", when(col("teamPointsPerc") === col("bestOfTeam"), 1).otherwise(0))
    .withColumn("dominationPerc", round(sum(col("dominatedTeammate")).over(driverWindow) / count(col("year")).over(driverWindow) * 100, 2))
    .withColumn("totalSeasons", count(col("year")).over(driverWindow))
    .where(col("totalSeasons") > 4)
    .select("year", "driverId", "dominatedTeammate", "dominationPerc")

driverDomination: DataFrame = [year: string, driverId: string ... 2 more fields]

In [12]:
val driverFilter = spark.read.parquet("../../../data/parquetnopart/results.parquet")
    .withColumn("finished", when(col("statusId") === 1, 1).otherwise(0))
    .withColumn("numberOfFinishes", sum(col("finished")).over(driverWindow))
    .where(col("numberOfFinishes") < 5)
    .select("driverId")
    .distinct

driverFilter: Dataset[Row] = [driverId: string]

In [13]:
val raceConstructorWindow = Window.partitionBy("raceId", "constructorId")
val seasonConstructorWindow = Window.partitionBy("year", "constructorId")

raceConstructorWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2440b853
seasonConstructorWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6cebe166

In [14]:
val teammateComparison = spark.read.parquet("../../../data/parquetnopart/results.parquet")
    .join(driverFilter, Seq("driverId"), "leftanti")
//     .where(not(col("driverId").isInCollection(driverFilter)))
    .join(races, "raceId")


    .withColumn("position", col("position").cast(IntegerType))
    .withColumn("grid", col("grid").cast(IntegerType))
    .na.fill(Map("position" -> 100, "grid" -> 100))

    .withColumn("grid", when(col("grid") === 0, 100).otherwise(col("grid")))

    .withColumn("topPos", min(col("position")).over(raceConstructorWindow))
    .withColumn("constructorBestPos", when(col("topPos") === col("position"), 1).otherwise(0))
    .withColumn("topPosPerc", sum(col("constructorBestPos")).over(driverSeasonWindow) / 
                count(col("raceId")).over(driverSeasonWindow) * 100)

    .withColumn("constTopPosPerc", max(col("topPosPerc")).over(seasonConstructorWindow))
    .withColumn("driverDomConstPos", when(col("constTopPosPerc") === col("topPosPerc"), 1).otherwise(0))

    .withColumn("topGrid", min(col("grid")).over(raceConstructorWindow))
    .withColumn("constructorBestGridPos", when(col("topGrid") === col("grid"), 1).otherwise(0))
    .withColumn("topGridPerc", sum(col("constructorBestGridPos")).over(driverSeasonWindow) / 
                count(col("raceId")).over(driverSeasonWindow) * 100)

    .withColumn("constTopGridPerc", max(col("topGridPerc")).over(seasonConstructorWindow))
    .withColumn("driverDomConstGrid", when(col("constTopGridPerc") === col("topGridPerc"), 1).otherwise(0))

    .dropDuplicates("driverId", "year")

    .withColumn("avgTopPosPerc", avg(col("topPosPerc")).over(driverWindow))
    .withColumn("avgTopGridPerc", avg(col("topGridPerc")).over(driverWindow))
    .withColumn("avgPosDom", avg(col("driverDomConstPos")).over(driverWindow))
    .withColumn("avgGridDom", avg(col("driverDomConstGrid")).over(driverWindow))


    .dropDuplicates("driverId")

//     .select("year", "driverId","topPosPerc", "driverDomConstPos","topGridPerc", "driverDomConstGrid")
    .select("driverId", "avgTopPosPerc", "avgTopGridPerc", "avgPosDom", "avgGridDom")
    .sort(col("avgPosDom").desc)

teammateComparison: Dataset[Row] = [driverId: string, avgTopPosPerc: double ... 3 more fields]

In [15]:
val averageRank = udf((cols: Seq[Int]) => {cols.sum / cols.size}: Double)
spark.udf.register("averageRank", averageRank)

averageRank: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd14$Helper$$Lambda$4863/1341021492@2c2f7db3,
  DoubleType,
  List(
    Some(
      ExpressionEncoder(
        NewInstance(
          class org.apache.spark.sql.catalyst.util.GenericArrayData,
          List(
            BoundReference(0, ObjectType(interface scala.collection.Seq), true)
          ),
          List(),
          true,
          ArrayType(IntegerType, false),
          None
        ),
        UnresolvedMapObjects(
          org.apache.spark.sql.catalyst.ScalaReflection$$$Lambda$4876/289421155@5975e81b,
          GetColumnByOrdinal(0, ArrayType(IntegerType, false)),
          Some(interface scala.collection.Seq)
        ),
        scala.collection.Seq
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"scala

In [16]:
val results = spark.read.parquet("../../../data/parquetnopart/results.parquet")
    .join(driverFilter, Seq("driverId"), "left_anti")
    .join(races, "raceId")
//     .join(driverInfo, "driverId")
    
    .withColumn("grid", col("grid").cast(IntegerType))
    .withColumn("position", col("position").cast(IntegerType))

    // porcentaje de salidas en primera fila (primero o segundo)
    .withColumn("firstRowStart", when(col("grid") === 1  || col("grid") === 2, 1).otherwise(0))
    .withColumn("firstRowChance", round(sum(col("firstRowStart")).over(driverWindow) / count(col("firstRowStart")).over(driverWindow), 4) * 100)

    // porcentaje de temporadas en las que ha superado a su compañero de equipo
    .join(driverDomination, Seq("driverId", "year"), "left")
    .join(teammateComparison, Seq("driverId"), "left")

    // posicion media en parrilla
    .withColumn("avgGridStart", round(avg(col("grid")).over(driverWindow), 2))
    // posicion media al acabar la carrera
    .withColumn("avgFinish", round(avg(col("position")).over(driverWindow), 2))
    
    // porcentaje de temporadas con pole
    .withColumn("pole", when(col("grid") === 1, 1).otherwise(0))
    .withColumn("totalPolePositions", sum(col("pole")).over(driverWindow))
    .withColumn("poleChance", round(col("totalPolePositions") / count(col("pole")).over(driverWindow), 4) * 100)
    .withColumn("polesPerSeason", sum(col("pole")).over(driverSeasonWindow))
    .withColumn("poleChance", round(col("totalPolePositions") / count(col("raceId")).over(driverWindow) * 100, 2))
    .withColumn("hasPoleThisSeason", when(col("polesPerSeason") > 0, 1).otherwise(0))
    .withColumn("percSeasonsWithPole", round(sum(col("hasPoleThisSeason")).over(driverWindow) / count(col("year")).over(driverWindow), 4) * 100)
    
    // porcentaje de temporadas con victoria
    .withColumn("win", when(col("position") === 1, 1).otherwise(0))
    .withColumn("totalVictories", sum(col("win")).over(driverWindow))
    .withColumn("victoryChance", round(col("totalVictories") / count(col("win")).over(driverWindow), 4) * 100)
    .withColumn("winsPerSeason", sum(col("win")).over(driverSeasonWindow))
    .withColumn("hasWonThisSeason", when(col("winsPerSeason") > 0, 1).otherwise(0))
    .withColumn("percSeasonsWithWins", round(sum(col("hasWonThisSeason")).over(driverWindow) / count(col("year")).over(driverWindow), 4) * 100)

    // porcentaje de podios
    .withColumn("podium", when(col("position") === 1 || col("position") === 2 ||col("position") === 3, lit(1)).otherwise(lit(0)))
    .withColumn("podiumChance", round(sum(col("podium")).over(driverWindow) / count(col("podium")).over(driverWindow), 4) * 100)
    

    .dropDuplicates("driverId")
    .select("driverId", "firstRowChance", "avgGridStart", "avgFinish", "totalPolePositions", "poleChance", "percSeasonsWithPole", "percSeasonsWithWins", "podiumChance", "dominationPerc", "avgTopPosPerc", "avgTopGridPerc", "avgPosDom", "avgGridDom")
    .withColumn("rankFRC", rank().over(Window.orderBy(col("firstRowChance").desc)))
    .withColumn("rankAGS", rank().over(Window.orderBy(col("avgGridStart").asc)))
    .withColumn("rankAF", rank().over(Window.orderBy(col("avgFinish").asc)))
    .withColumn("rankTPP", rank().over(Window.orderBy(col("totalPolePositions").desc)))
    .withColumn("rankPSWP", rank().over(Window.orderBy(col("percSeasonsWithPole").desc)))
    .withColumn("rankPSWW", rank().over(Window.orderBy(col("percSeasonsWithWins").desc)))
    .withColumn("rankPC", rank().over(Window.orderBy(col("podiumChance").desc)))
    .withColumn("rankDom", rank().over(Window.orderBy(col("dominationPerc").desc)))
    .withColumn("rankPoleC", rank().over(Window.orderBy(col("poleChance").desc)))
    .withColumn("rankPosPerc", rank().over(Window.orderBy(col("avgTopPosPerc").desc)))
    .withColumn("rankGridPerc", rank().over(Window.orderBy(col("avgTopGridPerc").desc)))
    .withColumn("rankPosDom", rank().over(Window.orderBy(col("avgPosDom").desc)))
    .withColumn("rankGridDom", rank().over(Window.orderBy(col("avgGridDom").desc)))
    .withColumn("stats", averageRank(
        array(col("rankFRC"),
            col("rankAGS"),
            col("rankAF"),
            col("rankTPP"),
            col("rankPSWP"),
            col("rankPSWW"),
            col("rankPC"),
            col("rankDom"),
            col("rankPoleC"),
            col("rankPosPerc"),
            col("rankGridPerc"),
            col("rankPosDom"),
            col("rankGridDom")
           )
    ))
    .withColumn("rank", rank().over(Window.orderBy(col("stats").asc)))
    .sort(col("rank").asc)
    .join(driverInfo, "driverId")

results: DataFrame = [driverId: string, firstRowChance: double ... 28 more fields]

In [17]:
def run[A](code: => A): Long = {
    val start = System.currentTimeMillis()
    val res = code
    System.currentTimeMillis() - start
}

def average(list: List[Long]):Long = list.sum / list.size.toLong

def timeTest(f: => Long): Long = {
    val list = (1 to 1).map(_ => f).toList
    average(list)
}

val res = timeTest(run(results.collect))

defined function run
defined function average
defined function timeTest
res: Long = 3770L